<a href="https://colab.research.google.com/github/miotomita/covid_automation/blob/main/Vdata%E7%94%A8%E3%81%AE%E4%BA%BA%E5%8F%A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [127]:
!pip install tabula-py
!pip install gspread_pandas
#gspreadのモジュールエラーが出たため（2021/12/30）
!git clone https://github.com/robin900/gspread-dataframe.git
%cd gspread-dataframe
!python setup.py install

Cloning into 'gspread-dataframe'...
remote: Enumerating objects: 634, done.
remote: Counting objects: 100% (298/298), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 634 (delta 195), reused 200 (delta 103), pack-reused 336
Receiving objects: 100% (634/634), 143.16 KiB | 9.54 MiB/s, done.
Resolving deltas: 100% (378/378), done.
/content/gspread-dataframe/gspread-dataframe/gspread-dataframe
running install
running bdist_egg
running egg_info
creating gspread_dataframe.egg-info
writing gspread_dataframe.egg-info/PKG-INFO
writing dependency_links to gspread_dataframe.egg-info/dependency_links.txt
writing requirements to gspread_dataframe.egg-info/requires.txt
writing top-level names to gspread_dataframe.egg-info/top_level.txt
writing manifest file 'gspread_dataframe.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'gspread_dataframe.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_6

In [128]:
import pandas as pd
import tabula
import re
import numpy as np
from decimal import Decimal, ROUND_HALF_EVEN,ROUND_HALF_UP,ROUND_HALF_DOWN, ROUND_CEILING, ROUND_FLOOR,ROUND_UP, ROUND_DOWN, ROUND_05UP

In [129]:
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [130]:
#認証
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [297]:
#日本ワクチンBD
japan_vax = gc.open_by_key('1U2-Kwgr08sC1mP8rwgW8U4NyYuuX2VQYhhG-nS9BtQM')
#日本BD、改定後シート（予定、2021.12末時点）
japan = gc.open_by_key('1sdkJZ-h-k3ywr_bpWDkRae0ExDfSXwG5RdOSzNtcdqk')

In [274]:
pref_names = pd.read_csv('https://raw.githubusercontent.com/miotomita/meta/main/prefecture_names.csv',index_col=0)
pref_code_dic = pref_names.loc[~pref_names.code_pref.isna()].set_index('prefecture_j').code_pref.to_dict()

In [132]:
def round0(x):
    return float(Decimal(str(x)).quantize(Decimal('0'),rounding=ROUND_HALF_UP))

In [133]:
def roundceil(x):
  return float(Decimal(str(x)).quantize(Decimal('0'),rounding=ROUND_CEILING))

In [134]:
def roundfloor(x):
  return float(Decimal(str(x)).quantize(Decimal('0'),rounding=ROUND_FLOOR))

# 年齢階級別人口、住民基本台帳

In [135]:
#【総計】令和3年住民基本台帳年齢階級別人口（市区町村別）
juki_url = 'https://www.soumu.go.jp/main_content/000762465.xlsx'

In [136]:
juki_raw = pd.read_excel(juki_url,header=1)

In [137]:
juki_raw.columns = ['コード','都道府県名','市区町村名','性別'] + juki_raw.columns[4:].to_list()

In [138]:
juki_raw['コード'] = juki_raw['コード'].apply(lambda x:pd.to_numeric(x,errors='ignore'))

# 市区町村データからの積み上げ

In [139]:
#市区町村コード
url = 'https://www.soumu.go.jp/main_content/000730858.xlsx'

In [140]:
codes = pd.read_excel(url,sheet_name='R1.5.1現在の団体')
codes2 = pd.read_excel(url, sheet_name = 'H30.10.1政令指定都市')

In [141]:
#現在の団体（都道府県＋市区町村）
codes = codes.iloc[:,:-2].replace({'合計':'全国','\s':'','\W':'',' ':''}, regex=True)
codes.columns = ['コード','都道府県名','市区町村名']
codes['コード'] = codes['コード'].apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [142]:
#政令市＋サブディビジョン
codes2 = codes2.iloc[:,:-2].replace({'合計':'全国','\s':'','\W':'',' ':''}, regex=True)
codes2.columns = ['コード','都道府県名','市区町村名']
codes2['コード'] = codes2['コード'].apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [143]:
#都道府県
pref_code = codes[codes.市区町村名.isna()].set_index('コード').dropna(how='all',axis=1)
pref_code.head()

,都道府県名
コード,
10006,北海道
20001,青森県
30007,岩手県
40002,宮城県
50008,秋田県


In [144]:
#基礎自治体
dist_code = codes[~codes.市区町村名.isna()].set_index('コード')
dist_code

,都道府県名,市区町村名
コード,,
11002,北海道,札幌市
12025,北海道,函館市
12033,北海道,小樽市
12041,北海道,旭川市
12050,北海道,室蘭市
...,...,...
473618,沖縄県,久米島町
473626,沖縄県,八重瀬町
473758,沖縄県,多良間村


In [145]:
#北方領土
kuril = [16951, 16969, 16977, 16985, 16993, 17001]
#東京、島嶼（小笠原などと重複）
isld = [133604]

In [146]:
#政令市
seirei = list(set(codes2.コード)&set(dist_code.index))
#政令市のサブディビジョン
seirei_sub = list(set(codes2.コード) - set(seirei))

In [147]:
#人口データのコードすべて - 北方領土・東京島嶼・政令市のサブ・基礎自治体
dist_na = list(set(juki_raw['コード'].apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna()) - set(kuril + isld + seirei_sub + dist_code.index.to_list()+pref_code.index.to_list()) )

In [148]:
#郡部
gunbu = juki_raw.loc[(juki_raw.コード.isin(dist_na))&(juki_raw.性別=='計')&(juki_raw.市区町村名.apply(lambda x: str(x)[-1]=='郡'))]['コード'].to_list()

In [149]:
#中芸広域連合＝398781
#高知県奈半利町、田野町、安田町、北川村、馬路村が連携しちゅう・・・・
#ここを先に合算しないと、12歳〜人口の全国合算が「１人」少なくなるため注意
chugei = [393029, 393037, 393045, 393053, 393061]
juki_raw.loc[juki_raw.コード.isin(chugei)&(juki_raw.性別=='計')]

,コード,都道府県名,市区町村名,性別,総数,0歳～4歳,5歳～9歳,10歳～14歳,15歳～19歳,20歳～24歳,25歳～29歳,30歳～34歳,35歳～39歳,40歳～44歳,45歳～49歳,50歳～54歳,55歳～59歳,60歳～64歳,65歳～69歳,70歳～74歳,75歳～79歳,80歳～84歳,85歳～89歳,90歳～94歳,95歳～99歳,100歳以上
5695,393029,高知県,安芸郡奈半利町,計,3119,82,86,92,113,105,102,104,123,141,154,146,206,251,287,326,218,225,205,94,37,8
5698,393037,高知県,安芸郡田野町,計,2606,81,95,72,80,86,74,125,116,127,135,136,197,188,215,239,189,187,144,75,31,6
5701,393045,高知県,安芸郡安田町,計,2594,72,85,61,77,75,95,71,105,114,149,138,184,208,215,248,194,209,167,92,30,1
5704,393053,高知県,安芸郡北川村,計,1247,20,35,46,52,40,45,48,55,67,63,57,77,85,100,123,87,86,100,41,17,0
5707,393061,高知県,安芸郡馬路村,計,832,18,28,36,23,37,38,31,43,24,51,43,63,61,55,63,67,63,57,21,7,1


In [150]:
set(dist_na) - set(gunbu)

set()

住民基本台帳、男女別、市区町村別

In [151]:
#男女別、基礎自治体
juki_mf = juki_raw.loc[(juki_raw.性別.isin(['男','女']))&(~juki_raw.コード.isin(pref_code.index.to_list()+gunbu+kuril+isld+seirei_sub+['-']))].reset_index(drop=True)

In [152]:
juki_mf.iloc[:,4:] = juki_mf.iloc[:,4:].applymap(lambda x: pd.to_numeric(x, errors='coerce'))

In [153]:
juki_mf.columns = [col.replace('～','から') for col in juki_mf.columns]

In [154]:
#ワクチン接種では中芸広域連合（高知）の５町村を合算してから計算する（デジタル庁の説明）
#http://www.chugei-kouiki.jp/
chugei_data = juki_mf.loc[juki_mf.コード.isin(chugei)].groupby(['性別']).sum()
chugei_data['都道府県名'] = '高知県'
chugei_data['市区町村名'] = '中芸広域連合'
chugei_data['コード'] = 398781
chugei_data = chugei_data.reset_index()
chugei_data

,性別,コード,都道府県名,市区町村名,総数,0歳から4歳,5歳から9歳,10歳から14歳,15歳から19歳,20歳から24歳,25歳から29歳,30歳から34歳,35歳から39歳,40歳から44歳,45歳から49歳,50歳から54歳,55歳から59歳,60歳から64歳,65歳から69歳,70歳から74歳,75歳から79歳,80歳から84歳,85歳から89歳,90歳から94歳,95歳から99歳,100歳以上
0,女,398781,高知県,中芸広域連合,5438,134,162,150,171,143,151,169,214,203,282,255,354,391,428,524,430,477,433,234,101,16
1,男,398781,高知県,中芸広域連合,4960,139,167,157,174,200,203,210,228,270,270,265,373,402,444,475,325,293,240,89,21,0


In [155]:
juki_mf = juki_mf.loc[~juki_mf.コード.isin(chugei)].append(chugei_data).reset_index(drop=True)

In [156]:
#合算する団体数
juki_mf.shape[0]/2

1737.0

In [157]:
juki_mf.tail()

,コード,都道府県名,市区町村名,性別,総数,0歳から4歳,5歳から9歳,10歳から14歳,15歳から19歳,20歳から24歳,25歳から29歳,30歳から34歳,35歳から39歳,40歳から44歳,45歳から49歳,50歳から54歳,55歳から59歳,60歳から64歳,65歳から69歳,70歳から74歳,75歳から79歳,80歳から84歳,85歳から89歳,90歳から94歳,95歳から99歳,100歳以上
3469,473812,沖縄県,八重山郡竹富町,女,2089,116,123,121,53,96,113,152,174,147,165,101,94,122,126,83,51,56,77,71,26,4
3470,473821,沖縄県,八重山郡与那国町,男,913,45,46,54,27,26,25,71,71,89,83,61,58,78,54,63,26,14,15,3,0,0
3471,473821,沖縄県,八重山郡与那国町,女,784,54,54,46,30,18,29,36,63,53,51,47,48,46,57,48,21,25,31,9,3,2
3472,398781,高知県,中芸広域連合,女,5438,134,162,150,171,143,151,169,214,203,282,255,354,391,428,524,430,477,433,234,101,16
3473,398781,高知県,中芸広域連合,男,4960,139,167,157,174,200,203,210,228,270,270,265,373,402,444,475,325,293,240,89,21,0


基礎データの取得はここまでで終わり<br>
12~14歳の計算用にはここまで↑の基礎自治体別・男女別データを使う

In [284]:
#都道府県別
juki_by_pref = juki_mf.fillna(0).groupby('都道府県名').sum().reindex(pref_code['都道府県名'].to_list()).astype(int).drop('コード',axis=1)
juki_by_pref.loc['全国'] = juki_by_pref.sum()
juki_by_pref.insert(1,'全人口_年齢性別判明分',juki_by_pref.loc[:,'0歳から4歳':'100歳以上'].sum(axis=1))

In [285]:
juki_by_pref['numeric'] = juki_by_pref.index.map(pref_code_dic)
juki_by_pref = juki_by_pref.reset_index(drop=False).set_index(['numeric','都道府県名'])

In [286]:
juki_by_pref.tail()

,,総数,全人口_年齢性別判明分,0歳から4歳,5歳から9歳,10歳から14歳,15歳から19歳,20歳から24歳,25歳から29歳,30歳から34歳,35歳から39歳,40歳から44歳,45歳から49歳,50歳から54歳,55歳から59歳,60歳から64歳,65歳から69歳,70歳から74歳,75歳から79歳,80歳から84歳,85歳から89歳,90歳から94歳,95歳から99歳,100歳以上
numeric,都道府県名,,,,,,,,,,,,,,,,,,,,,,,
44.0,大分県,1141784,1141741,40983,47365,50622,51249,51270,48096,53555,62567,71822,78693,68153,67895,74227,85431,94946,66562,55097,42777,22614,6801,1016
45.0,宮崎県,1087372,1087241,42167,49123,51802,50522,44733,43534,49980,60174,67654,72499,63025,66581,74823,84170,88014,58543,51401,40247,20825,6358,1066
46.0,鹿児島県,1617850,1617517,63756,74054,76574,74200,64143,65320,76140,91952,98458,102543,94353,101639,115879,128892,122973,83714,76271,61238,32923,10772,1723
47.0,沖縄県,1485484,1485118,78778,85509,84897,81256,77134,78465,86277,96243,100390,108997,94765,89179,91255,96207,80111,50196,48321,33849,16704,5365,1220
0.0,全国,126654244,126645025,4621243,5211654,5486075,5718629,6376817,6442752,6775050,7597655,8503905,9920558,8801840,8008744,7412109,8098283,9337065,6890167,5356363,3702117,1782079,521284,80636


# VD用

#日本ワクチン

In [121]:
#この段階で四捨五入する
age12_14 = (juki_mf['10歳から14歳']/5*3).apply(lambda x: Decimal(str(x)).quantize(Decimal('0'), rounding=ROUND_HALF_UP)).astype(int)
age10_11 = (juki_mf['10歳から14歳']/5*2).apply(lambda x: Decimal(str(x)).quantize(Decimal('0'), rounding=ROUND_HALF_UP)).astype(int)

In [319]:
vac_age_raw = pd.concat([juki_mf.loc[:,'都道府県名'],
                         #年代属性別
                         juki_mf.loc[:,'0歳から4歳':'100歳以上'].sum(axis=1).rename('全人口'),
                         (age10_11 + juki_mf.loc[:,'0歳から4歳':'5歳から9歳'].sum(axis=1)).rename('12歳未満'),
                         (age12_14 + juki_mf.loc[:,'15歳から19歳':].sum(axis=1)).rename('12歳以上'),
                         (age12_14 + juki_mf.loc[:,'15歳から19歳':'60歳から64歳'].sum(axis=1)).rename('12歳から64歳'),
                         (juki_mf.loc[:,'0歳から4歳':'60歳から64歳'].sum(axis=1)).rename('64歳以下'),
                         juki_mf.loc[:,'65歳から69歳':].sum(axis=1).rename('65歳以上'),
                         #年齢別接種用
                         #12～19歳	20代	30代	40代	50代	60～64歳	65～69歳	70代	80代	90代	100歳以上
                         (age12_14 + juki_mf.loc[:,'15歳から19歳']).rename('12歳から19歳'),
                         (juki_mf.loc[:,'20歳から24歳'] + juki_mf.loc[:,'25歳から29歳']).rename('20代'),
                         (juki_mf.loc[:,'30歳から34歳'] + juki_mf.loc[:,'35歳から39歳']).rename('30代'),
                         (juki_mf.loc[:,'40歳から44歳'] + juki_mf.loc[:,'45歳から49歳']).rename('40代'),
                         (juki_mf.loc[:,'50歳から54歳'] + juki_mf.loc[:,'55歳から59歳']).rename('50代'),
                         juki_mf.loc[:,'60歳から64歳'],
                         juki_mf.loc[:,'65歳から69歳'],
                         (juki_mf.loc[:,'70歳から74歳'] + juki_mf.loc[:,'75歳から79歳']).rename('70代'),
                         (juki_mf.loc[:,'80歳から84歳'] + juki_mf.loc[:,'85歳から89歳']).rename('80代'),
                         (juki_mf.loc[:,'90歳から94歳'] + juki_mf.loc[:,'95歳から99歳']).rename('90代'),
                         juki_mf.loc[:,'100歳以上']],axis=1).fillna(0)

In [320]:
#都道府県別
vac_age = vac_age_raw.groupby('都道府県名').sum().reindex(pref_code['都道府県名'].to_list()).astype(int)
vac_age.loc['全国'] = vac_age.sum()

In [321]:
vac_age['numeric'] = vac_age.index.map(pref_code_dic)
vac_age = vac_age.reset_index(drop=False).set_index(['numeric','都道府県名'])

In [322]:
vac_age.tail()

,,全人口,12歳未満,12歳以上,12歳から64歳,64歳以下,65歳以上,12歳から19歳,20代,30代,40代,50代,60歳から64歳,65歳から69歳,70代,80代,90代,100歳以上
numeric,都道府県名,,,,,,,,,,,,,,,,,
44.0,大分県,1141741,108597,1033144,657900,766497,375244,81622,99366,116122,150515,136048,74227,85431,161508,97874,29415,1016
45.0,宮崎県,1087241,112010,975231,624607,736617,350624,81604,88267,110154,140153,129606,74823,84170,146557,91648,27183,1066
46.0,鹿児島県,1617517,168440,1449077,930571,1099011,518506,120144,129463,168092,201001,195992,115879,128892,206687,137509,43695,1723
47.0,沖縄県,1485118,198243,1286875,954902,1153145,331973,132197,155599,182520,209387,183944,91255,96207,130307,82170,22069,1220
0.0,全国,126645025,12027309,114617716,78849722,90877031,35767994,9010292,12819569,14372705,18424463,16810584,7412109,8098283,16227232,9058480,2303363,80636


In [323]:
#日本ワクチンの「人口」シートを更新
set_with_dataframe(japan_vax.worksheet('人口'),vac_age,include_index=True,resize=False)

#日本BD

In [324]:
age_group_raw = pd.concat([juki_mf.loc[:,'都道府県名'],
                           juki_mf.loc[:,'0歳から4歳':].sum(axis=1).rename('全人口'),
                           (juki_mf['0歳から4歳'] + juki_mf['5歳から9歳']).rename('10歳未満'),
                           (juki_mf['10歳から14歳'] + juki_mf['15歳から19歳']).rename('10代'),
                           (juki_mf['20歳から24歳'] + juki_mf['25歳から29歳']).rename('20代'),
                           (juki_mf['30歳から34歳'] + juki_mf['35歳から39歳']).rename('30代'),
                           (juki_mf['40歳から44歳'] + juki_mf['45歳から49歳']).rename('40代'),
                           (juki_mf['50歳から54歳'] + juki_mf['55歳から59歳']).rename('50代'),
                           (juki_mf['60歳から64歳'] + juki_mf['65歳から69歳']).rename('60代'),
                           (juki_mf['70歳から74歳'] + juki_mf['75歳から79歳']).rename('70代'),
                           (juki_mf['80歳から84歳'] + juki_mf['85歳から89歳']).rename('80代'),
                           (juki_mf['90歳から94歳'] + juki_mf['95歳から99歳']).rename('90代'),
                           juki_mf.loc[:,'100歳以上'],
                           juki_mf.loc[:,'80歳から84歳':'100歳以上'].sum(axis=1).rename('80歳以上')],axis=1).fillna(0)

In [325]:
#都道府県別
age_group = age_group_raw.groupby('都道府県名').sum().reindex(pref_code['都道府県名'].to_list()).astype(int)
age_group.loc['全国'] = age_group.sum()

In [326]:
age_group['numeric'] = age_group.index.map(pref_code_dic)
age_group = age_group.reset_index(drop=False).set_index(['numeric','都道府県名'])

In [327]:
#Vdata用に順番並べ替え
age_group[age_group.columns[:-5].to_list()+['80歳以上']+age_group.columns[-5:-1].to_list()].tail()

,,全人口,10歳未満,10代,20代,30代,40代,50代,60代,80歳以上,70代,80代,90代,100歳以上
numeric,都道府県名,,,,,,,,,,,,,
44.0,大分県,1141741,88348,101871,99366,116122,150515,136048,159658,128305,161508,97874,29415,1016
45.0,宮崎県,1087241,91290,102324,88267,110154,140153,129606,158993,119897,146557,91648,27183,1066
46.0,鹿児島県,1617517,137810,150774,129463,168092,201001,195992,244771,182927,206687,137509,43695,1723
47.0,沖縄県,1485118,164287,166153,155599,182520,209387,183944,187462,105459,130307,82170,22069,1220
0.0,全国,126645025,9832897,11204704,12819569,14372705,18424463,16810584,15510392,11442479,16227232,9058480,2303363,80636


In [329]:
#日本BDの「人口」シートに書き込み
set_with_dataframe(japan.worksheet('人口'),age_group[age_group.columns[:-5].to_list()+['80歳以上']+age_group.columns[-5:-1].to_list()].tail(),include_index=True,resize=False)

# Vdata automation用のcsv

In [336]:
age_automation = pd.concat([juki_by_pref, vac_age.add_suffix('_VD_vac'),age_group.add_suffix('_VD_jp')],axis=1)

In [337]:
age_automation.to_csv('age_vdata.csv')

In [338]:
from google.colab import files
files.download('age_vdata.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [334]:
age_automation

,,総数,全人口_年齢性別判明分,0歳から4歳,5歳から9歳,10歳から14歳,15歳から19歳,20歳から24歳,25歳から29歳,30歳から34歳,35歳から39歳,40歳から44歳,45歳から49歳,50歳から54歳,55歳から59歳,60歳から64歳,65歳から69歳,70歳から74歳,75歳から79歳,80歳から84歳,85歳から89歳,90歳から94歳,95歳から99歳,100歳以上,全人口_VDワクチン,12歳未満_VDワクチン,12歳以上_VDワクチン,12歳から64歳_VDワクチン,64歳以下_VDワクチン,65歳以上_VDワクチン,12歳から19歳_VDワクチン,20代_VDワクチン,30代_VDワクチン,40代_VDワクチン,50代_VDワクチン,60歳から64歳_VDワクチン,65歳から69歳_VDワクチン,70代_VDワクチン,80代_VDワクチン,90代_VDワクチン,100歳以上_VDワクチン,全人口_VD日本,10歳未満_VD日本,10代_VD日本,20代_VD日本,30代_VD日本,40代_VD日本,50代_VD日本,60代_VD日本,70代_VD日本,80代_VD日本,90代_VD日本,100歳以上_VD日本,80歳以上_VD日本
numeric,都道府県名,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,北海道,5228732,5226603,163053,191324,206274,220374,229785,231610,252800,293077,339246,394320,351510,345760,338612,394130,429532,307187,246784,175632,86277,25458,3858,5226603,436880,4789723,3120865,3557745,1668858,344145,461395,545877,733566,697270,338612,394130,736719,422416,111735,3858,5226603,354377,426648,461395,545877,733566,697270,732742,736719,422416,111735,3858,538009
2.0,青森県,1260067,1259615,38423,44932,49399,55055,49729,49263,55090,67662,78115,88559,84192,86858,91769,102630,106429,73827,62831,47140,21412,5601,699,1259615,103112,1156503,735934,839046,420569,84697,98992,122752,166674,171050,91769,102630,180256,109971,27013,699,1259615,83355,104454,98992,122752,166674,171050,194399,180256,109971,27013,699,137683
3.0,岩手県,1221205,1220823,37893,45814,50623,54419,49133,49345,54967,66008,76300,84080,77426,80933,85933,96420,97298,70160,63518,48522,24328,6787,916,1220823,103959,1116864,708915,812874,407949,84790,98478,120975,160380,158359,85933,96420,167458,112040,31115,916,1220823,83707,105042,98478,120975,160380,158359,182353,167458,112040,31115,916,144071
4.0,宮城県,2282106,2281989,80065,93466,98615,104783,113335,112247,124393,143432,158880,171908,148703,142556,145175,161016,164186,112709,93041,68356,34218,9547,1358,2281989,212976,2069013,1424582,1637558,644431,163953,225582,267825,330788,291259,145175,161016,276895,161397,43765,1358,2281989,173531,203398,225582,267825,330788,291259,306191,276895,161397,43765,1358,206520
5.0,秋田県,971604,971288,25524,31730,36488,38987,34081,33424,39486,49499,59121,63783,58882,65032,73522,84357,87202,59851,55985,44765,22803,5999,767,971288,71849,899439,537710,609559,361729,60880,67505,88985,122904,123914,73522,84357,147053,100750,28802,767,971288,57254,75475,67505,88985,122904,123914,157879,147053,100750,28802,767,130319
6.0,山形県,1070017,1069562,34381,40941,45351,48889,44153,42242,49288,58422,66704,70141,64305,68965,75794,84874,86238,59829,54339,43293,23876,6657,880,1069562,93464,976098,616112,709576,359986,76098,86395,107710,136845,133270,75794,84874,146067,97632,30533,880,1069562,75322,94240,86395,107710,136845,133270,160668,146067,97632,30533,880,129045
7.0,福島県,1862777,1862059,63108,72110,78300,87314,83797,83031,91866,104138,116998,127928,115694,122398,132834,146606,142924,98822,84159,64912,34212,9575,1333,1862059,166536,1695523,1112980,1279516,582543,134296,166828,196004,244926,238092,132834,146606,241746,149071,43787,1333,1862059,135218,165614,166828,196004,244926,238092,279440,241746,149071,43787,1333,194191
8.0,茨城県,2907678,2907675,99487,116832,127230,135415,142372,138356,150808,171708,192666,224605,196758,180326,180475,208311,225456,164008,121475,78868,39414,11489,1616,2907675,267211,2640464,1789827,2057038,850637,211753,280728,322516,417271,377084,180475,208311,389464,200343,50903,1616,2907675,216319,262645,280728,322516,417271,377084,388786,389464,200343,50903,1616,252862
9.0,栃木県,1955402,1955401,67909,79400,86250,90898,92866,93318,103268,117803,132771,151767,130418,121530,124172,141813,151658,102788,77135,53427,27122,8034,1054,1955401,181809,1773592,1210561,1392370,563031,142648,186184,221071,284538,251948,124172,141813,254446,130562,35156,1054,1955401,147309,177148,186184,221071,284538,251948,265985,254446,130562,35156,1054,166772
